# Jupyter notebook de avaliação dos dados da produção da Brazilian FISH

In [1]:
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import xlsxwriter
from tabulate import tabulate
import seaborn as sns

#from google.colab import drive
#from electre_tree import tree_e_tri_b , util_e_tri_b
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.preprocessing import MinMaxScaler
#from yellowbrick.cluster import KElbowVisualizer
#from yellowbrick.cluster import SilhouetteVisualizer
# Required Libraries
import os


In [2]:
dataset = pd.DataFrame()
file = pd.ExcelFile('data\C_DADOS_BZF.xlsx')
sheets_list = ['Produção', 'Industria', 'Descrição']
for sheet in sheets_list:
    if 'Descrição' != sheet:
        temp_df = pd.read_excel(file, sheet_name=sheet)
        dataset = pd.concat([dataset, temp_df])
first_list = temp_df.columns.tolist()
temp_df = dataset.reset_index(drop=True)
#dataset.head()
len(dataset)

4504

In [3]:
xls = pd.ExcelFile('data\C_DADOS_BZF.xlsx')
prod= pd.read_excel(xls, 'Produção')
ind= pd.read_excel(xls, 'Industria')

In [6]:
## Eliminar espacos dos nomes das colunas
prod.columns = prod.columns.str.strip()

In [9]:
prod_num=prod.select_dtypes(include='number')


Peso Estimado Biometria           1722
Biomassa Biometria                1720
Peso Médio Últ. Biometria         1720
Peso Médio Biometria              1720
Biomassa Despesca                 1703
Peso Médio Despesca               1703
Densidade                         1703
Peso Médio Atual                  1703
Custo Alevinos                      26
Custo Ração                         26
Custo Insumos                       26
Custo/Kg                            26
Custos Totais (Peixe)               26
Custos Totais (Biomassa)            26
Custo/Peixe                         26
Ração Real                          26
% Nominal/Real                      26
GPD Real                            21
Ração Nominal                       20
Conversão Alimentar                  6
Custo Entrada/Unidades               0
Custo Total Entrada (Biomassa)       0
Custo Indireto                       0
Custo Entrada/Kg                     0
Custo Total Entrada (Peixe)          0
ID Lote                  

In [10]:
prod.select_dtypes(include='object').head()

,Unidade,Tanque,Fase,Tamanho,Lote,Data Encerramento,Curva,Status,Espécie,Tipo Tanque,Linha,Produto Origem,Fornecedor Origem
0,Unidade 3,TR111,Fase 3,P,L055-VAC,2022-03-29 00:00:00,F3 VERÃO - Engorda 100%,Encerrado,Tilápia,Engorda,U.3 - LINHA 10,Avulsa,Avulsa
1,Unidade 3,TR142,Fase 3,M,L055-VAC,2022-03-07 00:00:00,F3 VERÃO - Engorda 100%,Encerrado,Tilápia,Engorda,U.3 - LINHA 09,Avulsa,Avulsa
2,Unidade 3,TR254,Fase 3,G,L055-VAC,2022-02-17 00:00:00,F3 VERÃO - Engorda 100%,Encerrado,Tilápia,Engorda,U.3 - LINHA 03,Avulsa,Avulsa
3,Unidade 3,TR123,Fase 3,M,L057-VAC,2022-03-08 00:00:00,F3 VERÃO - Engorda 100%,Encerrado,Tilápia,Engorda,NaN,Avulsa,Avulsa
4,Unidade 3,TR670,Fase 4,G,L048-VAC,2022-03-08 00:00:00,F4 VERÃO - Engorda 100%,Encerrado,Tilápia,Engorda,U.3 - LINHA 07,Avulsa,Avulsa


In [11]:
prod_cat=prod.select_dtypes(include='object')

Linha                474
Unidade                0
Tanque                 0
Fase                   0
Tamanho                0
Lote                   0
Data Encerramento      0
Curva                  0
Status                 0
Espécie                0
Tipo Tanque            0
Produto Origem         0
Fornecedor Origem      0
dtype: int64

In [12]:
prod_num = prod_num.drop(columns=[
'Peso Estimado Biometria',
'Biomassa Biometria',
'Peso Médio Últ. Biometria',
'Peso Médio Biometria',
'Biomassa Despesca',
'Peso Médio Despesca',
'Densidade',
'Peso Médio Atual',
'Custo Insumos',
'Custo Alevinos',
'Custo Indireto',
'Densidade Máxima', 
'Sequência'
'Outras Saídas',
'Biomassa Atual', 
'Saldo'
])

prod_num.head()

,ID Lote,nº Dias,Peso Médio Inicial,Saldo,Quantidade Inicial,Mortalidade,Mortalidade Coletada,Sobrevivência,Saldo Final,Outras Saídas,...,Custo Alevinos,Custo Indireto,Custos Totais (Peixe),Custos Totais (Biomassa),Custo/Kg,Custo/Peixe,Ração Nominal,Ração Real,% Nominal/Real,Sequência
0,7201,85,46.0,0,8089,2254,204,72.184448,5839,0,...,0.0,0,24010.032110,19125.523013,7.737645,4.112011,5279.906040,4239.251772,80.290288,6
1,7202,63,84.0,0,8176,1549,235,81.054305,6627,0,...,0.0,0,22633.531848,19979.966837,6.981896,3.415351,4359.112894,3821.237976,87.660909,7
2,7203,45,188.0,0,8516,1307,98,84.652419,7209,0,...,0.0,0,17717.644222,21469.799126,7.118032,2.457712,3704.695446,2902.144641,78.336929,13
3,7206,63,80.0,0,8152,1590,157,80.495584,6562,0,...,0.0,0,22797.248557,19860.435832,7.318226,3.474131,4298.687760,3828.892759,89.071200,0
4,7207,63,590.0,0,8100,328,328,96.604938,7825,0,...,0.0,0,75240.713679,57577.926096,8.175779,9.615427,6845.448715,6354.039134,92.821368,15


In [13]:
#prod_num1=prod_num.iloc[:,:-20]

#prod_num2=prod_num.iloc[:,-20:]

In [ ]:
dados = prod_num.copy()

tabela = tabulate(dados, headers="firstrow", tablefmt="sheet")

dados.to_excel('prod_num.xlsx', index = False)